<a href="https://colab.research.google.com/github/Vlasovets/Deep_learning_course_assistantship/blob/master/mouse_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x # Oleg's code

In [0]:
from google.colab import drive
drive.mount('/content/drive/MyDrive') # Oleg's code

In [0]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.models import Sequential, save_model
from tensorflow.python.keras.layers import Dense, LSTM, Bidirectional, Dropout
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [23]:
# Some GPUs require setting the `allow_growth` setting.
# Comment out this code is you don't have a GPU card.
import tensorflow.compat.v1 as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config=config)

In [24]:
print(tf.__version__) # Oleg's code

1.15.0


In [0]:
def load_dataset(filename):
    X, y = [], []
    with open(filename) as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter='\t')
        for row in csv_reader:
            moves = parse_moves(row['movements'])
            age = parse_age(row['age'])
            X.append(moves)
            y.append(age)
    X, y = np.array(X), np.array(y)
    return X, y

In [0]:
def parse_moves(seq):
    result = []
    coords = seq.split(',')
    for coord in coords:
        x, y, t = coord.split(';')
        result.append([int(x), int(y)])
    return result


In [0]:
def parse_age(age):
    if age == 'young':
        return 0
    elif age == 'adult':
        return 1
    elif age == 'elder':
        return 2

In [0]:
def create_model(shape):
    units = shape[0]
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=shape))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:
from google.colab import files # Oleg's code
uploaded = files.upload()

Saving mousemoves.csv to mousemoves.csv


In [33]:
if __name__ == '__main__':
    # The saved model file will be named like the dataset file.
    dataset_file = 'mousemoves.csv'

    X, y = load_dataset(dataset_file)

    # All sequences must have the same length.
    X = pad_sequences(X)

    # Create partitions.
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

    # Set callbacks, for monitoring progress.
    cb_tensorboard = TensorBoard(log_dir='/tmp/mouse_logs')
    cb_earlystopping = EarlyStopping(patience=10)
    cb_checkpoint = ModelCheckpoint('/tmp/mouse_logs/best.h5', save_best_only=True)

    # Train the model.
    model = create_model(X_train[0].shape)
    print(model.summary())
    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_test, y_test),
        epochs=100,
        callbacks=[cb_tensorboard, cb_earlystopping, cb_checkpoint]
    )

    # Evaluate the model.
    loss, acc = model.evaluate(X_test, y_test)
    print('ACC: {:.2f}'.format(acc))

    # Save the model.
    save_model(model, '{}.h5'.format(dataset_file))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 444)               399600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 444)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 1335      
Total params: 400,935
Trainable params: 400,935
Non-trainable params: 0
_________________________________________________________________
None
Train on 204 samples, validate on 51 samples
Epoch 1/100
204/204 [==============================] - 9s 45ms/sample - loss: 1.2412 - acc: 0.2990 - val_loss: 1.1029 - val_acc: 0.3529
Epoch 2/100
204/204 [==============================] - 6s 31ms/sample - loss: 1.1536 - acc: 0.3627 - val_loss: 1.1036 - val_acc: 0.3922
Epoch 3/100
204/204 [==============================] - 6